# ML.Net - Multi_Class_Classifier_LBFGSMaximumEntropy

In [1]:
// ML.NET Nuget packages 
#r "nuget:Microsoft.ML"  
    
//Install XPlot package
#r "nuget:XPlot.Plotly"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package XPlot.Plotly version 3.0.1

Installed package Microsoft.ML version 1.5.0

In [2]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using Microsoft.ML.Transforms;
using static Microsoft.ML.DataOperationsCatalog;
using XPlot.Plotly;

In [3]:
//Step 1: Declare Path to Training Data Set
static readonly string _trainingDataPath = Path.Combine(Environment.CurrentDirectory, "./Datasets/Iris_data/iris-data.csv");

//Declare Path to Saving the Trained MLModel
static readonly string _modelPath = Path.Combine(Environment.CurrentDirectory, "./Datasets/Iris_data/Multi_Class_Classifier_LBFGSMaximumEntropy/Model.zip");

In [4]:
class IrisInputDataModel
{
    [LoadColumn(0)]
    public float SepalLength;

    [LoadColumn(1)]
    public float SepalWidth;

    [LoadColumn(2)]
    public float PetalLength;

    [LoadColumn(3)]
    public float PetalWidth;

    [LoadColumn(4)]
    public string Label;
}

class IrisOutputDataModel
{
    [ColumnName("PredictedLabel")]
    public string PredictedLabels;

}

In [5]:
//Step 2: Create MLContext() object
 MLContext mlContext = new MLContext();

 //Step 3: Create IDataView() object using mlContext.Data() to load Training Data Set
 IDataView trainingDataView = mlContext.Data.LoadFromTextFile<IrisInputDataModel>(path: _trainingDataPath, hasHeader: false, separatorChar: ',');

 // Step 4: Create dataPreparationPipeline() using mlContext.Tranforms(), Map Value to Key "Label", and Concatenate() the four 
 //Feature Columns - "SepalLength, "SepalWidth", "PetalLength", "PetalWidth"
 string featuresColumnName = "Features";

 var dataPreparationPipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
     .Append(mlContext.Transforms.Concatenate(featuresColumnName, "SepalLength", "SepalWidth", "PetalLength", "PetalWidth"))
     .AppendCacheCheckpoint(mlContext); //Do NOT use CacheCheckPoint for large Data Sets - might affect performance

 //Step 5: Create lbfgsMaxEntropyMultiClass Classification Trainer using dataPreparationPipeline() object
 var lbfgsMaxEntropyMultiClassTrainer = dataPreparationPipeline.Append(mlContext.MulticlassClassification.Trainers.LbfgsMaximumEntropy(labelColumnName: "Label", featureColumnName: featuresColumnName))
     .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

 // Step 6: Split Training Data Set trainingDataView() into 80% for Training and 20% for Evaluation using TrainTestData() object
 TrainTestData trainTestData = mlContext.Data.TrainTestSplit(trainingDataView, testFraction: 0.2);

 //Step 7: Create Trained MLModel - model using lbfgsMaxEntropyMultiClassTrainer.Fit()
 //Pass trainTestData.TrainSet() for Training MLModel
 var model = lbfgsMaxEntropyMultiClassTrainer.Fit(trainTestData.TrainSet);

 //Evaluate Trained MLModel
 //Step 8: Use mlContext.MultiClassClassification.Evaluate() Method to obtain Evaluation metrics 
 //Pass trainTestData.TestSet() to model.Trainsforms() method to calculate metrics
 var metrics = mlContext.MulticlassClassification.Evaluate(model.Transform(trainTestData.TestSet));

 // Step 9: Display the Performance Metrics of the MLModel
 Console.WriteLine();
 Console.WriteLine("Multi-Class Classification Algorithm: LBFGSMaximumEntropy");
 Console.WriteLine("  Assessing MLModel Quality using Evaluation Metrics");
 Console.WriteLine("--------------------------------------------------");
 Console.WriteLine();
 Console.WriteLine($"  Confusion Matrix: {metrics.ConfusionMatrix.GetFormattedConfusionTable().ToString()}");
 Console.WriteLine();
 Console.WriteLine($"  Macro Accuracy: {metrics.MacroAccuracy:P2}");
 Console.WriteLine($"  Micro Accuracy: {metrics.MicroAccuracy:P2}");
 Console.WriteLine();
 Console.WriteLine($"  Log Loss: {metrics.LogLoss.ToString()}");
 Console.WriteLine($"  Log Loss Reduction: {metrics.LogLossReduction}");
 Console.WriteLine();
 Console.WriteLine($"  Top K Accuracy: {metrics.TopKAccuracy:P2}");
 Console.WriteLine($"  Top K Prediction Count: {metrics.TopKPredictionCount}");
 Console.WriteLine();
 Console.WriteLine("---------------End of Model Quality Metrics------------");

 //Step 10: Create prediction Data object that you want the Trained MLModel to Classify
 IrisInputDataModel predictionData = new IrisInputDataModel()
 {
     SepalLength = 4.5f,
     SepalWidth = 1.6f,
     PetalLength = 3.5f,
     PetalWidth = 5.1f,
 };

 //Step 11:  Create Predictor using Model.CreatePredictionEngine(model)
 var predictor = mlContext.Model.CreatePredictionEngine<IrisInputDataModel, IrisOutputDataModel>(model);

 //Step 12:  Now we want ot use the predictionEngine to predict the class of flower
 var predictedClassOfFlower = predictor.Predict(predictionData);

 //Step 13:  Wrting to the Output
 Console.WriteLine($"Predeicted Class of the Flower:  {string.Join("", predictedClassOfFlower.PredictedLabels.ToString())} ");
 Console.WriteLine("");

 // Step 14: Saving Trained MLModel for Developing Intelligent Apps
 using (var fileStream = new FileStream(_modelPath, FileMode.Create, FileAccess.Write, FileShare.Write))
 {
     mlContext.Model.Save(model, trainingDataView.Schema, fileStream);
     Console.WriteLine();
     Console.WriteLine("Trained Model has been saved in the specified Path");
     Console.WriteLine("------------End of Output----------------------------------");
 }


Multi-Class Classification Algorithm: LBFGSMaximumEntropy
  Assessing MLModel Quality using Evaluation Metrics
--------------------------------------------------

  Confusion Matrix: 
Confusion table
                   ||========================
PREDICTED          ||     0 |     1 |     2 | Recall
TRUTH              ||========================
0.     Iris-setosa ||     8 |     0 |     0 | 1,0000
1. Iris-versicolor ||     0 |     7 |     0 | 1,0000
2.  Iris-virginica ||     0 |     0 |     6 | 1,0000
                   ||========================
Precision          ||1,0000 |1,0000 |1,0000 |


  Macro Accuracy: 100,00%
  Micro Accuracy: 100,00%

  Log Loss: 0,13536419617319012
  Log Loss Reduction: 0,876015843165999

  Top K Accuracy: 0,00%
  Top K Prediction Count: 0

---------------End of Model Quality Metrics------------
Predeicted Class of the Flower:  Iris-virginica 



Unhandled exception: System.IO.DirectoryNotFoundException: Could not find a part of the path 'D:\Onderive\Particular\OneDrive\Projetos\ML.Net\MLDotNet_D_atual\Jupyter_Notebooks\Datasets\Iris_data\Multi_Class_Classifier_LBFGSMaximumEntropy\Model.zip'.
   at System.IO.FileStream.ValidateFileHandle(SafeFileHandle fileHandle)
   at System.IO.FileStream.CreateFileOpenHandle(FileMode mode, FileShare share, FileOptions options)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share, Int32 bufferSize, FileOptions options)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share)
   at Submission#11.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, 